In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install category_encoders

     |████████████████████████████████| 81kB 7.6MB/s 


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Lasso
from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import mean_squared_log_error, r2_score, mean_squared_error
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import seaborn as sns

SEED = 42

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [ ]:
cd gdrive/MyDrive

/content/gdrive/MyDrive


In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [ ]:
train.columns

Index(['ID', 'Price', 'Levy', 'Manufacturer', 'Model', 'Prod. year',
       'Category', 'Leather interior', 'Fuel type', 'Engine volume', 'Mileage',
       'Cylinders', 'Gear box type', 'Drive wheels', 'Doors', 'Wheel', 'Color',
       'Airbags'],
      dtype='object')

In [ ]:
train.dtypes

ID                    int64
Price                 int64
Levy                 object
Manufacturer         object
Model                object
Prod. year            int64
Category             object
Leather interior     object
Fuel type            object
Engine volume        object
Mileage              object
Cylinders           float64
Gear box type        object
Drive wheels         object
Doors                object
Wheel                object
Color                object
Airbags               int64
dtype: object

In [ ]:
#Numerical

#ID, Price, Prod. year, Cylinders, Airbags

In [ ]:
len(train.drop_duplicates())

18924

In [ ]:
len(train)

19237

In [ ]:
train.drop_duplicates(inplace = True)

In [ ]:
len(train)

18924

In [ ]:
train.info()
train.sample(10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18924 entries, 0 to 19236
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                18924 non-null  int64  
 1   Price             18924 non-null  int64  
 2   Levy              18924 non-null  object 
 3   Manufacturer      18924 non-null  object 
 4   Model             18924 non-null  object 
 5   Prod. year        18924 non-null  int64  
 6   Category          18924 non-null  object 
 7   Leather interior  18924 non-null  object 
 8   Fuel type         18924 non-null  object 
 9   Engine volume     18924 non-null  object 
 10  Mileage           18924 non-null  object 
 11  Cylinders         18924 non-null  float64
 12  Gear box type     18924 non-null  object 
 13  Drive wheels      18924 non-null  object 
 14  Doors             18924 non-null  object 
 15  Wheel             18924 non-null  object 
 16  Color             18924 non-null  object

,ID,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
12488,45816125,314,1017,HYUNDAI,Elantra,2017,Sedan,Yes,Petrol,2,110456 km,4.0,Automatic,Front,04-May,Left wheel,Silver,12
14087,44783767,18500,730,KIA,RIO,2016,Sedan,No,Petrol,1.6,53000 km,4.0,Tiptronic,Front,04-May,Left wheel,Silver,10
13143,45800974,15500,-,SKODA,Octavia,2013,Sedan,No,Petrol,1.8 Turbo,200000 km,4.0,Manual,Front,04-May,Left wheel,White,6
17662,45773578,23521,584,MERCEDES-BENZ,C 250,2014,Sedan,Yes,Petrol,1.8,64270 km,4.0,Automatic,Rear,04-May,Left wheel,Grey,0
7644,45760499,7500,-,SKODA,Octavia,2009,Hatchback,No,Petrol,2,305000 km,4.0,Manual,Front,04-May,Left wheel,White,4
3537,45645920,392,781,TOYOTA,Camry,2012,Sedan,Yes,Hybrid,2.5,314373 km,4.0,Automatic,Front,04-May,Left wheel,Grey,12
64,45803923,20526,640,HYUNDAI,Tucson,2013,Jeep,Yes,Diesel,2,200279 km,4.0,Automatic,Front,04-May,Left wheel,Grey,4
1716,45725107,18826,707,SSANGYONG,Actyon,2011,Jeep,Yes,Diesel,2,181000 km,4.0,Automatic,Front,04-May,Left wheel,Silver,4
7356,45803125,13297,707,HYUNDAI,Sonata,2011,Sedan,Yes,Petrol,2,149000 km,4.0,Automatic,Front,04-May,Left wheel,Silver,4
7257,45732551,31361,862,BMW,328,2013,Sedan,Yes,Petrol,2.8 Turbo,108800 km,4.0,Tiptronic,Rear,04-May,Left wheel,White,4


In [ ]:
train[['Mileage']] = train[['Mileage']].replace(to_replace=r'([a-z/]+|[A-Z/]+)', value='', regex=True)
train["Mileage"] = train["Mileage"].astype("str").astype("int")

test[['Mileage']] = test[['Mileage']].replace(to_replace=r'([a-z/]+|[A-Z/]+)', value='', regex=True)
test["Mileage"] = test["Mileage"].astype("str").astype("int")

In [ ]:
train.head()

,ID,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
0,45654403,13328,1399,LEXUS,RX 450,2010,Jeep,Yes,Hybrid,3.5,186005,6.0,Automatic,4x4,04-May,Left wheel,Silver,12
1,44731507,16621,1018,CHEVROLET,Equinox,2011,Jeep,No,Petrol,3,192000,6.0,Tiptronic,4x4,04-May,Left wheel,Black,8
2,45774419,8467,-,HONDA,FIT,2006,Hatchback,No,Petrol,1.3,200000,4.0,Variator,Front,04-May,Right-hand drive,Black,2
3,45769185,3607,862,FORD,Escape,2011,Jeep,Yes,Hybrid,2.5,168966,4.0,Automatic,4x4,04-May,Left wheel,White,0
4,45809263,11726,446,HONDA,FIT,2014,Hatchback,Yes,Petrol,1.3,91901,4.0,Automatic,Front,04-May,Left wheel,Silver,4


In [ ]:
train["Cylinders"] = train["Cylinders"].astype(int)
test["Cylinders"] = test["Cylinders"].astype(int)

In [ ]:
train['Levy'] = train['Levy'].replace("-", "0")
test['Levy'] = test['Levy'].replace("-", "0")

In [ ]:
train["Levy"] = train["Levy"].astype("str").astype("int")
test["Levy"] = test["Levy"].astype("str").astype("int")

In [ ]:
train.sample(10)

,ID,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
14948,45656281,314,586,TOYOTA,Prius,2012,Hatchback,Yes,Hybrid,1.8,418667,4,Automatic,Front,04-May,Left wheel,White,12
4381,45800848,15053,0,BMW,530,2004,Sedan,Yes,Petrol,3,200000,6,Tiptronic,Rear,04-May,Left wheel,White,0
5220,45788151,11917,0,NISSAN,Tiida,2006,Sedan,No,LPG,1.6,200000,4,Automatic,Front,04-May,Left wheel,Silver,2
14561,45706554,25716,0,MERCEDES-BENZ,Sprinter,1998,Microbus,No,Diesel,2.9 Turbo,290000,5,Manual,Rear,04-May,Left wheel,White,12
19199,45761409,3763,0,MERCEDES-BENZ,C 180,1995,Sedan,No,Petrol,1.8,1111111111,5,Manual,Rear,04-May,Right-hand drive,Green,2
3590,45783468,6586,0,OPEL,Vectra,1998,Sedan,No,Petrol,1.6,195256,4,Automatic,Front,04-May,Left wheel,Blue,4
3694,45803429,16308,891,HYUNDAI,Sonata,2016,Sedan,Yes,LPG,2,310064,4,Automatic,Front,04-May,Left wheel,White,4
12633,45758696,9722,574,HONDA,Insight,2010,Hatchback,No,Hybrid,1.3,183000,4,Variator,Front,04-May,Left wheel,Blue,9
11514,45809231,14269,1017,JAGUAR,F-pace,2017,Jeep,Yes,Diesel,2,50531,4,Automatic,4x4,04-May,Left wheel,Red,12
6856,45797107,13796,836,HYUNDAI,Genesis,2010,Coupe,Yes,Petrol,2,170000,4,Automatic,Front,04-May,Left wheel,Black,4


In [ ]:
#Label Encoder

lb = LabelEncoder()

train["Leather interior"] = lb.fit_transform(train["Leather interior"])
test["Leather interior"] = lb.fit_transform(test["Leather interior"])

In [ ]:
train["Drive wheels"] = lb.fit_transform(train["Drive wheels"])
test["Drive wheels"] = lb.fit_transform(test["Drive wheels"])

In [ ]:
train["Doors"] = lb.fit_transform(train["Doors"])
test["Doors"] = lb.fit_transform(test["Doors"])

In [ ]:
train["Wheel"] = lb.fit_transform(train["Wheel"])
test["Wheel"] = lb.fit_transform(test["Wheel"])

In [ ]:
train["Color"] = lb.fit_transform(train["Color"])
test["Color"] = lb.fit_transform(test["Color"])

In [ ]:
train.head()

,ID,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
0,45654403,13328,1399,LEXUS,RX 450,2010,Jeep,1,Hybrid,3.5,186005,6,Automatic,0,1,0,12,12
1,44731507,16621,1018,CHEVROLET,Equinox,2011,Jeep,0,Petrol,3,192000,6,Tiptronic,0,1,0,1,8
2,45774419,8467,0,HONDA,FIT,2006,Hatchback,0,Petrol,1.3,200000,4,Variator,1,1,1,1,2
3,45769185,3607,862,FORD,Escape,2011,Jeep,1,Hybrid,2.5,168966,4,Automatic,0,1,0,14,0
4,45809263,11726,446,HONDA,FIT,2014,Hatchback,1,Petrol,1.3,91901,4,Automatic,1,1,0,12,4


In [ ]:
train["Category"].unique()

array(['Jeep', 'Hatchback', 'Sedan', 'Microbus', 'Goods wagon',
       'Universal', 'Coupe', 'Minivan', 'Cabriolet', 'Limousine',
       'Pickup'], dtype=object)

In [ ]:
train["Category"] = lb.fit_transform(train["Category"])
test["Category"] = lb.fit_transform(test["Category"])

In [ ]:
train["Manufacturer"] = lb.fit_transform(train["Manufacturer"])
test["Manufacturer"] = lb.fit_transform(test["Manufacturer"])

In [ ]:
train["Model"] = lb.fit_transform(train["Model"])
test["Model"] = lb.fit_transform(test["Model"])

In [ ]:
train.sample(10)

,ID,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
10011,45771861,1568,1493,36,601,2016,9,1,Diesel,3.5,77205,6,Automatic,2,1,0,7,12
15029,45720188,15681,562,8,537,2015,9,1,Petrol,1.4 Turbo,20800,4,Automatic,2,1,0,1,6
13521,45792214,6500,0,61,1120,1999,9,1,Petrol,1.8 Turbo,3000,4,Manual,1,1,0,1,6
3315,43790286,17000,2236,35,288,2015,9,1,Diesel,2.2,59000,4,Tiptronic,1,1,1,1,8
9265,45809124,17023,501,21,684,2014,3,1,Petrol,1.5,52350,4,Automatic,1,1,0,12,4
16719,44927509,17249,475,8,537,2012,9,1,Petrol,1.4 Turbo,122000,4,Tiptronic,1,1,0,14,6
8479,45789708,11604,0,42,281,2007,2,0,Diesel,1.7,173000,4,Manual,1,1,0,2,2
5452,45793617,19757,0,55,1567,2014,4,0,Petrol,2,180000,4,Tiptronic,0,1,0,8,12
7876,45797556,34419,1249,23,834,2017,10,1,Diesel,2.5,184189,4,Automatic,1,1,0,12,4
17933,44851403,17876,0,42,861,2011,9,1,Petrol,2,180000,4,Tiptronic,1,1,0,14,4


In [ ]:
train["Fuel type"] = lb.fit_transform(train["Fuel type"])
test["Fuel type"] = lb.fit_transform(test["Fuel type"])

In [ ]:
train["Gear box type"] = lb.fit_transform(train["Gear box type"])
test["Gear box type"] = lb.fit_transform(test["Gear box type"])

In [ ]:
#encoder=ce.OneHotEncoder(cols=["Engine volume"], handle_unknown='return_nan',return_df=True,use_cat_names=True)

In [ ]:
#train = encoder.fit_transform(train)
#test = encoder.fit_transform(test)

In [ ]:
train['Engine volume'] = train['Engine volume'].replace(['2.0 Turbo', '2.2 Turbo','3.0 Turbo','1.4 Turbo','1.5 Turbo', '1.6 Turbo','2.3 Turbo','2.8 Turbo','1.8 Turbo', '2.4 Turbo', '3.5 Turbo', '3.2 Turbo','1.3 Turbo','2.5 Turbo','1.9 Turbo', '4.4 Turbo', '4.7 Turbo', '0.2 Turbo','4.8 Turbo', '4.6 Turbo','1.2 Turbo','1.7 Turbo', '6.3 Turbo', '2.7 Turbo','2.9 Turbo', '4.0 Turbo','3.6 Turbo','3.7 Turbo','5.5 Turbo','2.1 Turbo','0.7 Turbo', '0.6 Turbo','1.0 Turbo', '4.5 Turbo', '0.8 Turbo', '4.2 Turbo', '3.1', '5.0 Turbo','5.7 Turbo','0.4 Turbo', '5.4 Turbo', '0.3 Turbo','1.1 Turbo'],'Turbo')
train['Engine volume'] = train['Engine volume'].replace(['3.5','3','1.3','2.5','2','1.8','2.4','4','1.6','3.3','4.7','1.5','4.4','3.6','2.3','2.2','1.4','5.5','3.2','3.8','4.6','1.2','5', '1.7', '2.9', '0.5','1.9','2.7','4.8','5.3','0.4','2.8','1.1','2.1','0.7','5.4','3.7','1','2.6','5.7','3.4','4.3','4.2','5.9','6.8','4.5','0.6','7.3','0.1','6.3','6.4','5.2','5.8','0.8', '6.7', '6.2', '0', '20', '0.3', '0.2','5.6', '6', '3.9', '0.9'],'Non-Turbo')

In [ ]:
train["Engine volume"].unique()

array(['Non-Turbo', 'Turbo'], dtype=object)

In [ ]:
l3 = {
    "Non-Turbo" : 0,
    "Turbo" : 1
}

In [ ]:
train["Engine volume"] = train["Engine volume"].map(l3)

In [ ]:
train.drop(["ID"], axis = 1, inplace = True)

In [ ]:
train.head()

,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
0,13328,1399,32,1242,2010,4,1,2,0,186005,6,0,0,1,0,12,12
1,16621,1018,8,658,2011,4,0,5,0,192000,6,2,0,1,0,1,8
2,8467,0,21,684,2006,3,0,5,0,200000,4,3,1,1,1,1,2
3,3607,862,16,661,2011,4,1,2,0,168966,4,0,0,1,0,14,0
4,11726,446,21,684,2014,3,1,5,0,91901,4,0,1,1,0,12,4


In [ ]:
X = train.iloc[ : ,  1: ]
Y = train.iloc[ : , : 1]

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = SEED, shuffle = True)

In [ ]:
regressor = RandomForestRegressor(random_state = SEED)
regressor.fit(X_train, Y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [ ]:
predicted = regressor.predict(X_test)

In [ ]:
np.sqrt(mean_squared_log_error(Y_test, predicted))

1.0629215500958016

In [ ]:
r2_score(Y_test, predicted)

-1.3707085912656614

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Lasso

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
pipeline = Pipeline([
                     ('scaler',StandardScaler()),
                     ('model',Lasso())
])

In [ ]:
search = GridSearchCV(pipeline,
                      {'model__alpha':np.arange(0.1,10,0.1)},
                      cv = 5, scoring="neg_mean_squared_error",verbose=3
                      )

In [ ]:
features = train.columns

In [ ]:
features

Index(['Price', 'Levy', 'Manufacturer', 'Model', 'Prod. year', 'Category',
       'Leather interior', 'Fuel type', 'Engine volume', 'Mileage',
       'Cylinders', 'Gear box type', 'Drive wheels', 'Doors', 'Wheel', 'Color',
       'Airbags'],
      dtype='object')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [ ]:
pipeline = Pipeline([
                     ('scaler',StandardScaler()),
                     ('model',Lasso())
])

In [ ]:
search = GridSearchCV(pipeline,
                      {'model__alpha':np.arange(0.1,10,0.1)},
                      cv = 5, scoring="neg_mean_squared_error",verbose=3
                      )

In [ ]:
search.fit(X_train,y_train)

In [ ]:
search.best_params_


{'model__alpha': 9.9}

In [ ]:
coefficients = search.best_estimator_.named_steps['model'].coef_

In [ ]:
coefficients

array([-1286.32599104,   921.56543276, -1573.90220544,  6075.45340968,
       -2338.17115255, -1676.92128415, -4174.02686639,  1438.26377853,
        -310.15278839,  2578.29814634,  4153.60295299, -1174.80963821,
       -8967.54725674, -3504.09533254, -2899.28567091, -5518.46013485])

In [ ]:
importance = np.abs(coefficients)

In [ ]:
importance 

array([1286.32599104,  921.56543276, 1573.90220544, 6075.45340968,
       2338.17115255, 1676.92128415, 4174.02686639, 1438.26377853,
        310.15278839, 2578.29814634, 4153.60295299, 1174.80963821,
       8967.54725674, 3504.09533254, 2899.28567091, 5518.46013485])

In [ ]:
train.columns

Index(['Price', 'Levy', 'Manufacturer', 'Model', 'Prod. year', 'Category',
       'Leather interior', 'Fuel type', 'Engine volume', 'Mileage',
       'Cylinders', 'Gear box type', 'Drive wheels', 'Doors', 'Wheel', 'Color',
       'Airbags'],
      dtype='object')

In [ ]:
X_train

,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
16270,891,23,1305,2016,4,1,1,0,195996,4,0,1,1,0,14,4
15364,0,42,1037,2006,10,0,5,0,222222222,4,2,1,1,0,12,6
8071,1811,32,802,2010,4,1,5,0,155821,8,0,0,1,0,1,0
11766,934,16,742,2015,9,0,5,0,112000,5,0,1,1,0,14,8
12957,880,54,248,2018,4,1,1,0,38972,4,0,1,1,0,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11413,3989,24,1198,2006,4,1,5,0,219200,8,0,0,2,0,12,16
12106,1399,36,601,2010,9,1,1,0,256914,6,0,0,1,0,7,12
5429,0,5,68,1995,3,0,5,0,0,6,1,2,0,0,1,0
861,1018,5,97,2011,9,1,5,0,70000,6,2,2,1,0,14,12


In [ ]:
#Levy, Model  , Prod. year, Category, Leather interior, Fuel type, Engine volume, Cylinders, gear box type, drive wheels, doors, wheels, airbags

In [ ]:
X_train.drop(["Manufacturer"], axis = 1, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
X_test.drop(["Manufacturer"], axis = 1, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
X.drop(["Manufacturer"], axis =1 , inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = SEED, test_size = 0.25)

In [ ]:
regressor = RandomForestRegressor(random_state = SEED)
regressor.fit(X_train, Y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [ ]:
predicted = regressor.predict(X_test)

In [ ]:
r2_score(Y_test, predicted)

-23.734191919754963

In [ ]:
train

,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
0,13328,1399,32,1242,2010,4,1,2,0,186005,6,0,0,1,0,12,12
1,16621,1018,8,658,2011,4,0,5,0,192000,6,2,0,1,0,1,8
2,8467,0,21,684,2006,3,0,5,0,200000,4,3,1,1,1,1,2
3,3607,862,16,661,2011,4,1,2,0,168966,4,0,0,1,0,14,0
4,11726,446,21,684,2014,3,1,5,0,91901,4,0,1,1,0,12,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19232,8467,0,36,385,1999,1,1,0,1,300000,4,1,2,0,0,12,5
19233,15681,831,23,1334,2011,9,1,5,0,161600,4,2,1,1,0,11,8
19234,26108,836,23,1442,2010,4,1,1,0,116365,4,0,1,1,0,7,4
19235,5331,1288,8,456,2007,4,1,1,0,51258,4,0,1,1,0,1,4


In [ ]:
X = train.iloc[ : ,  1: ]
Y = train.iloc[ : , : 1]

In [ ]:
X

,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
0,1399,32,1242,2010,4,1,2,0,186005,6,0,0,1,0,12,12
1,1018,8,658,2011,4,0,5,0,192000,6,2,0,1,0,1,8
2,0,21,684,2006,3,0,5,0,200000,4,3,1,1,1,1,2
3,862,16,661,2011,4,1,2,0,168966,4,0,0,1,0,14,0
4,446,21,684,2014,3,1,5,0,91901,4,0,1,1,0,12,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19232,0,36,385,1999,1,1,0,1,300000,4,1,2,0,0,12,5
19233,831,23,1334,2011,9,1,5,0,161600,4,2,1,1,0,11,8
19234,836,23,1442,2010,4,1,1,0,116365,4,0,1,1,0,7,4
19235,1288,8,456,2007,4,1,1,0,51258,4,0,1,1,0,1,4


In [ ]:
Y

,Price
0,13328
1,16621
2,8467
3,3607
4,11726
...,...
19232,8467
19233,15681
19234,26108
19235,5331


In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestRegressor
 
from sklearn.linear_model import RidgeCV, Lasso, LassoCV
from sklearn.svm import LinearSVR
from sklearn import linear_model
import xgboost as xgb

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

SEED = 42

In [ ]:
estimators = [
              ("xgb", xgb.XGBRegressor()),
              ("svr", LinearSVR(random_state = SEED)),
              ("rf", RandomForestRegressor(random_state = SEED)),
            
]

In [ ]:
reg = StackingRegressor(
    estimators = estimators, 
    final_estimator = xgb.XGBRegressor(random_state = SEED)
)

In [ ]:
reg.fit(X_train, Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_stacking.py:642: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



[00:33:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.



[00:33:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:33:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:33:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:33:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:33:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.



[00:34:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


StackingRegressor(cv=None,
                  estimators=[('xgb',
                               XGBRegressor(base_score=0.5, booster='gbtree',
                                            colsample_bylevel=1,
                                            colsample_bynode=1,
                                            colsample_bytree=1, gamma=0,
                                            importance_type='gain',
                                            learning_rate=0.1, max_delta_step=0,
                                            max_depth=3, min_child_weight=1,
                                            missing=None, n_estimators=100,
                                            n_jobs=1, nthread=None,
                                            objective='reg:linear',
                                            random_state=0, reg_alpha=0,
                                            reg...
                                               colsample_bynode=1,
                          

In [ ]:
predict = reg.predict(X_test)

In [ ]:
r2_score(Y_test, predict)

0.5180597691901059

In [ ]:
test.drop(["ID", "Price"], axis = 1, inplace = True)

In [ ]:
test

,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
0,0,58,521,2012,3,0,1,2.0 Turbo,0,4,1,1,0,0,7,10
1,0,23,837,2012,9,1,5,2.4,26000,4,2,1,1,0,7,10
2,0,40,885,2005,9,0,5,1.5,168000,4,0,1,1,1,13,8
3,975,59,970,2012,4,1,5,3.2,143000,6,0,0,1,0,2,12
4,0,41,165,1993,3,0,5,1.6,200000,4,1,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8240,1357,15,867,2004,10,0,5,1.6,164000,4,1,1,1,0,2,6
8241,0,5,96,2012,9,1,5,2.8 Turbo,156000,4,2,1,1,0,1,6
8242,1646,30,509,2015,4,1,5,4.6,102907,8,0,0,1,0,1,0
8243,784,55,156,2017,3,1,5,1.5,3000,4,0,1,1,0,6,4


In [ ]:
test['Engine volume'] = test['Engine volume'].replace(['2.0 Turbo', '2.2 Turbo','3.0 Turbo','1.4 Turbo','1.5 Turbo', '1.6 Turbo','2.3 Turbo','2.8 Turbo','1.8 Turbo', '2.4 Turbo', '3.5 Turbo', '3.2 Turbo','1.3 Turbo','2.5 Turbo','1.9 Turbo', '4.4 Turbo', '4.7 Turbo', '0.2 Turbo','4.8 Turbo', '4.6 Turbo','1.2 Turbo','1.7 Turbo', '6.3 Turbo', '2.7 Turbo','2.9 Turbo', '4.0 Turbo','3.6 Turbo','3.7 Turbo','5.5 Turbo','2.1 Turbo','0.7 Turbo', '0.6 Turbo','1.0 Turbo', '4.5 Turbo', '0.8 Turbo', '4.2 Turbo', '3.1', '5.0 Turbo','5.7 Turbo','0.4 Turbo', '5.4 Turbo', '0.3 Turbo','1.1 Turbo', '6.0 Turbo', '2.6 Turbo'],'Turbo')
test['Engine volume'] = test['Engine volume'].replace(['3.5','3','1.3','2.5','2','1.8','2.4','4','1.6','3.3','4.7','1.5','4.4','3.6','2.3','2.2','1.4','5.5','3.2','3.8','4.6','1.2','5', '1.7', '2.9', '0.5','1.9','2.7','4.8','5.3','0.4','2.8','1.1','2.1','0.7','5.4','3.7','1','2.6','5.7','3.4','4.3','4.2','5.9','6.8','4.5','0.6','7.3','0.1','6.3','6.4','5.2','5.8','0.8', '6.7', '6.2', '0', '20', '0.3', '0.2','5.6', '6', '3.9', '0.9', '6.6', '6.1', '10.8'],'Non-Turbo')

In [ ]:
test["Engine volume"].unique()

array(['Turbo', 'Non-Turbo'], dtype=object)

In [ ]:
test["Engine volume"] = test["Engine volume"].map(l3)

In [ ]:
test["Engine volume"].unique()

array([1, 0])

In [ ]:
test.drop(["Manufacturer"], axis = 1, inplace = True)

In [ ]:
predict = reg.predict(test)

In [ ]:
len(predict)

8245

In [ ]:
len(test)

8245

In [ ]:
a = pd.read_csv("submission1.csv")

In [ ]:
a["Price"] = predict

In [ ]:
a.to_csv("submission1.csv", index = False)

In [ ]:
a

,Price
0,21554.597656
1,15943.228516
2,7054.545898
3,6802.278809
4,3659.496582
...,...
8240,9294.172852
8241,19083.671875
8242,12588.871094
8243,41458.980469


In [ ]:
scaler = StandardScaler()
print(scaler.fit(X_train))

StandardScaler(copy=True, with_mean=True, with_std=True)


In [ ]:
X_train.drop(["Doors"], axis = 1, inplace = True)

In [ ]:
test.drop(["Doors"], axis = 1, inplace = True)

In [ ]:
X_test.drop(["Doors"], axis = 1, inplace = True)

In [ ]:
reg.fit(X_train, Y_train)

[00:34:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_stacking.py:642: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.



[00:34:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:34:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:34:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:34:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:34:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.



[00:35:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


StackingRegressor(cv=None,
                  estimators=[('xgb',
                               XGBRegressor(base_score=0.5, booster='gbtree',
                                            colsample_bylevel=1,
                                            colsample_bynode=1,
                                            colsample_bytree=1, gamma=0,
                                            importance_type='gain',
                                            learning_rate=0.1, max_delta_step=0,
                                            max_depth=3, min_child_weight=1,
                                            missing=None, n_estimators=100,
                                            n_jobs=1, nthread=None,
                                            objective='reg:linear',
                                            random_state=0, reg_alpha=0,
                                            reg...
                                               colsample_bynode=1,
                          

In [ ]:
predict = reg.predict(X_test)

In [ ]:
r2_score(Y_test, predict)

0.4517486294164751

In [ ]:
X_test

,Levy,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Wheel,Color,Airbags
3499,751,1079,2013,9,1,5,0,180800,4,0,1,0,14,4
9232,751,1079,2013,9,1,5,0,230032,4,0,1,0,12,12
1810,765,1334,2015,9,1,5,0,14368,4,0,1,0,1,4
9625,0,1498,2001,3,0,5,0,155000,4,0,1,1,0,8
8522,0,1421,1998,6,0,1,1,111111,6,1,2,0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16876,1172,990,2011,4,1,5,0,129000,6,2,2,0,14,12
759,1624,601,2009,9,1,1,0,266061,6,0,0,0,1,12
5745,0,1154,2007,9,0,2,0,230000,8,0,1,0,14,8
11415,1325,1091,2019,4,1,5,0,15480,4,0,0,0,14,12


In [ ]:
X_train.drop(["Drive wheels", "Wheel"], axis = 1, inplace = True)
X_test.drop(["Drive wheels", "Wheel"], axis = 1, inplace = True)
test.drop(["Drive wheels", "Wheel"], axis = 1, inplace = True)

In [ ]:
reg.fit(X_train, Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_stacking.py:642: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



[00:35:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.



[00:35:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:35:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:35:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:35:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:35:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.



[00:35:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


StackingRegressor(cv=None,
                  estimators=[('xgb',
                               XGBRegressor(base_score=0.5, booster='gbtree',
                                            colsample_bylevel=1,
                                            colsample_bynode=1,
                                            colsample_bytree=1, gamma=0,
                                            importance_type='gain',
                                            learning_rate=0.1, max_delta_step=0,
                                            max_depth=3, min_child_weight=1,
                                            missing=None, n_estimators=100,
                                            n_jobs=1, nthread=None,
                                            objective='reg:linear',
                                            random_state=0, reg_alpha=0,
                                            reg...
                                               colsample_bynode=1,
                          

In [ ]:
predict = reg.predict(X_test)
r2_score(Y_test, predict)

-295.23855143087485